In [9]:
from zoology.model import LanguageModel
from zoology.experiments.d_model import configs

import torch
from torch.profiler import profile, record_function, ProfilerActivity

In [10]:
model = LanguageModel(configs[0].model)

In [13]:
model.to(7)
inputs = torch.randint(8_192, size=(64, 128)).to(7)

In [20]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)
prof.export_chrome_trace("trace.json")

STAGE:2023-10-27 20:36:36 4157758:4157758 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-10-27 20:36:36 4157758:4157758 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-10-27 20:36:36 4157758:4157758 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


In [19]:
print(prof.key_averages().table(sort_by="cpu_time_total"))

-------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                  model_inference        29.30%       9.368ms       100.00%      31.972ms      31.972ms             1  
                                  aten::embedding        12.41%       3.968ms        26.65%       8.519ms       8.519ms             1  
                               aten::index_select        13.90%       4.445ms        14.17%       4.531ms       4.531ms             1  
                                   aten::fft_rfft         5.10%       1.629ms         9.82%       3.140ms     196.250us            16  
                                    aten::dropou